# Model comparision for change localization with two features
- Experiment 3 in Shin & Ma (2015-2)

In [1]:
import fnmatch # file name matching
import os
from scipy.io import loadmat

from pandas import DataFrame, Series
import pandas as pd
import scipy as sp
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set('poster')
sns.set_style('whitegrid')

import statsmodels.api as sm

pd.options.mode.chained_assignment = None

# 1. Data 

In [6]:
# %load concat_matfiles.py
def concat_matfiles(exp_code,exp_ids,pathname,subject_names,exp_num=0):
    # Create a DataFrame from mat files by looping over subjects (subject_names)
    file_names = os.listdir(pathname)
    
    df = DataFrame()
    for exp_id in exp_ids:
        temp2 = DataFrame()
        for subject_name in subject_names:

            # Create a tag to find files based on a subject name
            if exp_num == 0:
                subject_tag = subject_name + str(exp_code) + '_' + str(exp_id) + '_'
            else:
                subject_tag = subject_name + str(exp_code) + '_' + str(exp_code) + '_' + str(exp_id) + '_'
            subject_files = fnmatch.filter(file_names, subject_tag + '*.mat') # Filter the files

            # Loop over filtered files
            temp1 = DataFrame()
            for subject_file in subject_files:

                # Convert a MATLAB matrix to a DataFrame
                temp0 = DataFrame(loadmat(pathname+subject_file)['datamatrix'])
                temp1 = pd.concat([temp1,temp0],ignore_index=True) # Concatenate DataFrames

            # Create a column to assign subject names
            temp1['Name'] = DataFrame([subject_name]*len(temp1))
            temp2 = pd.concat([temp2,temp1],ignore_index=True) # Concatenate

            # Create a column to assign experiment index
            if exp_id == 1:
                feat_name = 'OriA'
            elif exp_id == 2:
                feat_name = 'ColA'
            elif exp_id == 3:
                feat_name = 'OriC'
            elif exp_id == 4:
                feat_name = 'ColC'
            elif exp_id == 5:
                feat_name = 'OriD2'
            elif exp_id == 6:
                feat_name = 'ColD2'
            elif exp_code == 6 and exp_id == 1:
                feat_name = 'OriD1'
            elif exp_code == 6 and exp_id == 2:
                feat_name = 'ColD1'
            else:
                feat_name = 'B'                
            temp2['Feature'] = DataFrame([feat_name]*len(temp2))
                
        # Concatenate all subject data to create a single DataFrame            
        df = pd.concat([df,temp2],ignore_index=True)
    return df

In [5]:
def remap_colors(old_delta):
    if old_delta <= 180:
        new_delta = old_delta
    else:
        new_delta = 360 - old_delta
    new_delta = new_delta/2.0
    return new_delta

In [7]:
# Load data

pathnameD1 = 'one_change_lab/'
pathnameCD2 = 'two_feature_data/Exp3_all_change/' # this contains both Conditions A and D
subject_namesD1 = ['HJK','HS0','LLT','WK0','WZ0']
subject_namesCD2 = ['HJK','HS0','JSK','LLT','LN0','MBC','WK0','WZ0']

# there are only 3 subjects who did all experiments
common_subjects = set(subject_namesD1) & set(subject_namesCD2)
print common_subjects

set(['WK0', 'HS0', 'HJK', 'WZ0', 'LLT'])


In [8]:
exp_idsCD2 = [3,4,5,6] # 5: orientation + irrelevant color, 6: color + irrelevant orientation
exp_idsD1 = [1,2] # 1: orientation, 2: color
exp_codeCD2 = 3
exp_codeD1 = 6

# concatenating dataset
dfCD2 = concat_matfiles(exp_codeCD2,exp_idsCD2,pathnameCD2,common_subjects)
dfD1 = concat_matfiles(exp_codeD1,exp_idsD1,pathnameD1,common_subjects,exp_num=1)
df = pd.concat([dfCD2,dfD1],ignore_index=True)
df3 = df[['Name','Feature',1,2,8]]
df3.columns = ['Subject','Feature','Delta_Ori','Delta_Col','Correct']

df3.Delta_Col = df3.Delta_Col.apply((lambda x: remap_colors(x)))
df3.describe()

,Delta_Ori,Delta_Col,Correct
count,18000.000000,18000.000000,18000.000000
mean,-0.576351,30.277167,0.687500
std,42.391533,30.165285,0.463525
min,-90.000000,0.000000,0.000000
25%,-23.816156,0.000000,0.000000
50%,0.000000,23.000000,1.000000
75%,21.309192,57.000000,1.000000
max,90.000000,90.000000,1.000000


# 2. "Model 4"
- Orientation and color has its own independent resource.
- Entire resource is used to relevant information.
- A portion of resource is used to irrelevant information (ratio parameter).
- Each feature is independenlty encoded.
- In the decision-making step, an observer takes into account both relevant and irrelevant information.

In [ ]:
from scipy.special import i0
def compute_ds(J_bar,tau,target,delta,n_change):
    # J_bar = 0 is actually not possible because shape and scale parameters are defined in positive domain.
    Js = np.random.gamma(J_bar/tau, tau, N*2)
    k1 = Js[:N]
    k2 = Js[N:]
    
    dL_noise = np.random.vonmises(0, k1)-np.random.vonmises(0, k2)

    if n_change == 1:
        dL_noise[target] = dL_noise[target] + delta
    else:
        dL_noise = dL_noise + delta
        
    k_denom = np.sqrt(k1**2 + k2**2 + 2*k1*k2*np.cos(dL_noise))
    d = i0(k1)*i0(k2)*np.exp(k1+k2-k_denom)/i0(k_denom)

    return d